# 🔍 Similaridade de Cossenos vs. BM25: Potencializando Sistemas RAG com Busca Híbrida

Este notebook explica em detalhes a implementação de um sistema RAG (Retrieval-Augmented Generation) híbrido para consultas à LGPD, combinando a força semântica dos embeddings vetoriais com a precisão léxica do algoritmo BM25.

## 📊 Comparação de Técnicas de Busca

| Característica | Similaridade de Cossenos (Embeddings) | BM25 | Abordagem Híbrida |
|---------------|---------------------------------------|------|-------------------|
| Base matemática | Álgebra linear e espaço vetorial | Teoria da informação e estatística | Combinação ponderada |
| Captação semântica | ⭐⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐⭐ |
| Correspondência exata | ⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| Tratamento de termos raros | ⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |
| Sensibilidade a contexto | ⭐⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐⭐⭐ |
| Robustez a sinônimos | ⭐⭐⭐⭐ | ⭐ | ⭐⭐⭐⭐ |
| Requisitos computacionais | Alto (geração de embeddings) | Médio | Alto |

## 🔍 Por que BM25 + Similaridade de Cossenos É Superior

A busca híbrida combina o melhor de dois mundos: a **compreensão semântica de embeddings vetoriais** com a **precisão léxica do BM25**. Esta combinação resolve as limitações fundamentais de cada abordagem quando usada isoladamente.

### 🧩 Entendendo os Embeddings Vetoriais

Os embeddings vetoriais transformam texto em vetores de alta dimensão onde a similaridade de cossenos entre vetores representa a proximidade semântica entre os textos. Esta técnica é poderosa para capturar:

- **Relações semânticas**: Compreende que "proteção de dados" e "privacidade de informações" são conceitos relacionados
- **Contexto linguístico**: Captura o significado das palavras em seu contexto
- **Similaridade conceitual**: Identifica textos conceitualmente similares mesmo com vocabulário diferente

### ⚠️ Limitações da Similaridade de Cossenos

A similaridade de cossenos aplicada em embeddings vetoriais, apesar de poderosa, apresenta deficiências importantes:

1. **Problema dos Termos Raros**: Termos técnicos, nomenclaturas específicas ou entidades raras frequentemente são "diluídos" no espaço vetorial. Por exemplo, códigos de artigos específicos da LGPD podem não ser adequadamente representados.

2. **Correspondência Inexata**: Pode falhar ao não conseguir distinguir entre termos semanticamente próximos mas tecnicamente distintos. Por exemplo, "controlador de dados" e "operador de dados" são conceitos distintos na LGPD, mas podem ter embeddings próximos.

3. **Vocabulário Limitado**: Embeddings podem não representar adequadamente termos fora do vocabulário de treinamento, como terminologias jurídicas específicas da legislação brasileira.

4. **Fenômeno da "Média"**: Trechos longos acabam tendo suas características específicas atenuadas quando comprimidos em um único vetor, perdendo detalhes importantes em documentos jurídicos extensos.

### 📈 A Força do BM25

O algoritmo BM25 (Best Matching 25) é uma evolução sofisticada dos modelos estatísticos TF-IDF (Term Frequency-Inverse Document Frequency) que incorpora melhorias significativas:

1. **Saturação de Frequência**: Utiliza uma função de saturação que evita que termos repetidos dominem a pontuação, seguindo a intuição de que a relevância não aumenta linearmente com a frequência do termo.

2. **Normalização de Tamanho de Documento**: Ajusta pontuações para reduzir o viés em documentos mais longos, permitindo comparações mais justas entre trechos de diferentes tamanhos da LGPD.

3. **Valorização de Termos Raros**: Atribui pontuação mais alta para termos que raramente aparecem na coleção, destacando termos técnicos específicos da legislação.

4. **Correspondência Léxica Precisa**: Excelente para identificar termos técnicos e palavras-chave específicas, como "encarregado", "ANPD" ou "consentimento".

#### 🧮 A Fórmula do BM25

```
score(D,Q) = ∑(IDF(qi) × (f(qi,D) × (k1+1)) / (f(qi,D) + k1 × (1-b+b × |D|/avgdl)))
```

Onde:
- `D` é o documento (chunk de texto)
- `Q` é a consulta
- `qi` são os termos da consulta
- `f(qi,D)` é a frequência do termo no documento
- `|D|` é o tamanho do documento
- `avgdl` é o tamanho médio dos documentos na coleção
- `k1` e `b` são parâmetros de ajuste (tipicamente k1=1.2 e b=0.75)

### 💡 A Magia da Abordagem Híbrida

O sistema RAG implementado combina essas abordagens em uma busca híbrida através de uma fórmula de ponderação:

```
score_híbrido = α × score_embeddings + (1-α) × score_BM25
```

Onde `α` é um parâmetro de balanceamento (0.5 no código atual, representando peso igual para ambas as técnicas).

#### 📊 Fluxo de Recuperação Híbrida

1. **Consulta Paralela**: A consulta do usuário é processada simultaneamente pelo sistema de embeddings e pelo BM25
2. **União de Candidatos**: Os resultados de ambos os sistemas são combinados em um conjunto único de candidatos
3. **Reordenação Híbrida**: Cada documento candidato recebe uma pontuação combinada
4. **Seleção Final**: Os documentos com maior pontuação híbrida são selecionados

#### 🌟 Benefícios Tangíveis para RAG:

1. **Melhor Precisão para Termos Técnicos**: Legislação como a LGPD contém termos jurídicos específicos que o BM25 captura com precisão, como "titular de dados", "controlador" ou "operador".

2. **Maior Diversidade nas Respostas**: O BM25 recupera documentos que podem ser relevantes mas semanticamente distantes no espaço vetorial, enriquecendo o contexto fornecido ao LLM.

3. **Robustez a Consultas Complexas**: Consultas que mesclam conceitos gerais com termos específicos são melhor atendidas pela combinação de métodos.

4. **Complementaridade**: Uma técnica compensa as falhas da outra, criando um sistema mais resiliente a diferentes tipos de consultas.

5. **Menor Sensibilidade a Viés de Corpus**: A combinação reduz o efeito de enviesamento presente em cada método isolado.

6. **Adaptabilidade**: O parâmetro α pode ser ajustado para diferentes domínios ou tipos de consulta.

## 🚀 Boas Práticas e Observabilidade no Script RAG_BM25.py

O código implementa diversas boas práticas de desenvolvimento e observabilidade que merecem destaque:

### 🏗️ Estruturação e Modularização

1. **Separação de Responsabilidades**: O código segue o princípio de responsabilidade única com classes distintas:
   - `DocumentProcessor`: Extração e processamento de texto
   - `ChromaStore`: Gerenciamento do armazenamento vetorial
   - `LGPDRagSystem`: Orquestração do sistema RAG completo

2. **Modelos Pydantic com Validação**: Os modelos `SearchResult` e `LGPDResponse` implementam validações e documentação integrada, garantindo integridade dos dados.

3. **Tipagem Forte**: Uso extensivo de anotações de tipo (`typing`) para maior segurança e melhor documentação do código.

4. **Injeção de Dependências**: O padrão de injeção via `RagDependencies` facilita testes e manutenção.

### 📊 Observabilidade e Monitoramento

1. **Logging Detalhado**: O sistema utiliza o módulo `logging` para registrar cada etapa do processo com informações contextuais, facilitando depuração e monitoramento.

2. **Medição de Tempo**: Métricas de performance são coletadas para operações críticas:
   - Inicialização do sistema
   - Processamento de consultas
   - Busca de documentos relevantes

3. **Feedback ao Usuário**: A interface fornece informações sobre o progresso das operações, incluindo tempo de resposta e confiança.

4. **Rastreabilidade**: O sistema mantém identificação clara das fontes utilizadas para gerar respostas, essencial para auditoria em contextos jurídicos.

### 🛡️ Robustez e Tratamento de Erros

1. **Tratamento Abrangente de Exceções**: O código implementa captura de erros em diferentes níveis, garantindo degradação graciosa.

2. **Valores Padrão Sensatos**: Configurações via variáveis de ambiente com valores padrão bem definidos permitem operação imediata.

3. **Validações em Modelos**: Os modelos Pydantic garantem que dados estão dentro de parâmetros esperados, como confiança entre 0 e 1.

### 👤 Usabilidade e UX

1. **Prompts Estruturados**: O sistema fornece orientações claras para o LLM sobre como responder, incluindo diretrizes linguísticas específicas para o contexto jurídico brasileiro.

2. **Feedback Visual**: A formatação destaca elementos importantes como confiança e fontes, facilitando a interpretação dos resultados.

3. **Interface Conversacional Intuitiva**: A sessão interativa é projetada para facilitar a interação humana, com instruções claras e feedback contínuo.

### 🔌 Integrações Inteligentes

1. **PydanticAI para Ferramentas**: Uso de decoradores `@tool()` para facilitar integração com o framework de agentes.

2. **ChromaDB para Vetores**: Abstração eficiente do armazenamento vetorial com persistência.

3. **BM25 para Busca Léxica**: Implementação eficiente do algoritmo BM25 via biblioteca `rank-bm25`.

## 🏗️ Arquitetura do Sistema RAG Híbrido

O sistema implementa o fluxo RAG moderno com estas etapas bem definidas:

### 1️⃣ Ingestão e Processamento

- Extração de texto do PDF da LGPD usando PyPDF
- Divisão em chunks controlados com tamanho e sobreposição configuráveis
- Pré-processamento para otimizar recuperação

### 2️⃣ Indexação Dual

- **Vetorial**: Geração de embeddings e armazenamento no ChromaDB
- **Léxica**: Tokenização e construção do índice BM25
- Persistência de índices para reutilização

### 3️⃣ Recuperação Híbrida

- Processamento da consulta do usuário
- Busca paralela em ambos os índices
- Normalização de pontuações para escala comparável
- Fusão ponderada de resultados com parâmetro α configurável
- Seleção dos top-k documentos mais relevantes

### 4️⃣ Geração Aumentada

- Construção de prompt com contextos recuperados
- Instruções específicas para o LLM sobre formato e estilo de resposta
- Geração de resposta com base nos contextos fornecidos

### 5️⃣ Apresentação e Feedback

- Formatação estruturada da resposta
- Inclusão de metadados de confiança
- Citação das fontes utilizadas
- Métricas de desempenho

## 🧪 Experimentação e Ajustes

O sistema permite experimentação através de parâmetros configuráveis:

- **ALPHA**: Ajuste do peso entre embeddings e BM25 (0-1)
- **TOP_K**: Número de documentos a recuperar
- **CHUNK_SIZE** e **CHUNK_OVERLAP**: Controle da granularidade dos chunks
- **EMBEDDING_MODEL**: Seleção do modelo de embedding

## 🔮 Conclusão

Este sistema representa o estado da arte em recuperação de informações jurídicas, combinando o poder da compreensão semântica dos modelos de embedding com a precisão de correspondência textual do BM25. A abordagem híbrida supera as limitações de cada método isolado, resultando em respostas mais confiáveis e completas sobre a LGPD.

A implementação segue boas práticas de engenharia de software e oferece uma base sólida para aplicações de IA em contextos jurídicos, demonstrando como técnicas avançadas de recuperação de informação podem ser aplicadas para melhorar significativamente sistemas RAG.


## 💻 Exemplo de Uso do Sistema

Para utilizar o sistema RAG híbrido para consultas à LGPD, você pode executar o script principal ou importar o sistema em seu próprio código:

```python
# Importando o sistema
from RAG_BM25 import LGPDRagSystem
import asyncio

async def main():
    # Inicialização do sistema
    system = LGPDRagSystem()
    await system.initialize()
    
    # Exemplo de consulta
    response = await system.ask("Quais são os direitos do titular de dados na LGPD?")
    
    # Exibição da resposta
    print(f"Resposta (confiança: {response.confidence:.2f}):\n")
    print(response.answer)
    print("\nFontes utilizadas:")
    for source in response.sources:
        print(f"- {source}")

# Execução do exemplo
asyncio.run(main())
```

Este exemplo demonstra como inicializar o sistema e realizar uma consulta simples sobre os direitos do titular de dados na LGPD.